In [1]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.wsd import lesk
from typing import NamedTuple, Set, List, Dict

In [2]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')
#[nltk_data] Downloading package wordnet to /Users/ruhong/nltk_data...
#[nltk_data]   Unzipping corpora/wordnet.zip.
nltk.download('averaged_perceptron_tagger') # for pos tagging
nltk.data.path

[nltk_data] Downloading package wordnet to /home/pankun/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/pankun/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


['/home/pankun/nltk_data',
 '/mnt/s/dev/seahrh/nlp-examples/venv/nltk_data',
 '/mnt/s/dev/seahrh/nlp-examples/venv/share/nltk_data',
 '/mnt/s/dev/seahrh/nlp-examples/venv/lib/nltk_data',
 '/usr/share/nltk_data',
 '/usr/local/share/nltk_data',
 '/usr/lib/nltk_data',
 '/usr/local/lib/nltk_data']

In [3]:
wn.synsets("motorcar")

[Synset('car.n.01')]

In [4]:
wn.synset("car.n.01").lemma_names()

['car', 'auto', 'automobile', 'machine', 'motorcar']

In [5]:
wn.synset("car.n.01").definition()

'a motor vehicle with four wheels; usually propelled by an internal combustion engine'

In [6]:
for synset in wn.synsets("car"):
    print(f"{synset.name()}\n\t{synset.definition()}\n\tlemmas={synset.lemma_names()}")

car.n.01
	a motor vehicle with four wheels; usually propelled by an internal combustion engine
	lemmas=['car', 'auto', 'automobile', 'machine', 'motorcar']
car.n.02
	a wheeled vehicle adapted to the rails of railroad
	lemmas=['car', 'railcar', 'railway_car', 'railroad_car']
car.n.03
	the compartment that is suspended from an airship and that carries personnel and the cargo and the power plant
	lemmas=['car', 'gondola']
car.n.04
	where passengers ride up and down
	lemmas=['car', 'elevator_car']
cable_car.n.01
	a conveyance for passengers or freight on a cable railway
	lemmas=['cable_car', 'car']


In [7]:
for synset in wn.synsets("bank"):
    print(f"{synset.name()}\n\t{synset.definition()}\n\tlemmas={synset.lemma_names()}")

bank.n.01
	sloping land (especially the slope beside a body of water)
	lemmas=['bank']
depository_financial_institution.n.01
	a financial institution that accepts deposits and channels the money into lending activities
	lemmas=['depository_financial_institution', 'bank', 'banking_concern', 'banking_company']
bank.n.03
	a long ridge or pile
	lemmas=['bank']
bank.n.04
	an arrangement of similar objects in a row or in tiers
	lemmas=['bank']
bank.n.05
	a supply or stock held in reserve for future use (especially in emergencies)
	lemmas=['bank']
bank.n.06
	the funds held by a gambling house or the dealer in some gambling games
	lemmas=['bank']
bank.n.07
	a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force
	lemmas=['bank', 'cant', 'camber']
savings_bank.n.02
	a container (usually with a slot in the top) for keeping money at home
	lemmas=['savings_bank', 'coin_bank', 'money_box', 'bank']
bank.n.09
	a building in whi

# Hyponyms

In [8]:
motorcar = wn.synset("car.n.01")
print(motorcar.hyponyms())

[Synset('ambulance.n.01'), Synset('beach_wagon.n.01'), Synset('bus.n.04'), Synset('cab.n.03'), Synset('compact.n.03'), Synset('convertible.n.01'), Synset('coupe.n.01'), Synset('cruiser.n.01'), Synset('electric.n.01'), Synset('gas_guzzler.n.01'), Synset('hardtop.n.01'), Synset('hatchback.n.01'), Synset('horseless_carriage.n.01'), Synset('hot_rod.n.01'), Synset('jeep.n.01'), Synset('limousine.n.01'), Synset('loaner.n.02'), Synset('minicar.n.01'), Synset('minivan.n.01'), Synset('model_t.n.01'), Synset('pace_car.n.01'), Synset('racer.n.02'), Synset('roadster.n.01'), Synset('sedan.n.01'), Synset('sport_utility.n.01'), Synset('sports_car.n.01'), Synset('stanley_steamer.n.01'), Synset('stock_car.n.01'), Synset('subcompact.n.01'), Synset('touring_car.n.01'), Synset('used-car.n.01')]


In [9]:
tmp = []
for synset in motorcar.hyponyms():
    for lm in synset.lemma_names():
        tmp.append(lm)
tmp.sort()
print(f"{len(tmp)} hyponyms={tmp}")

66 hyponyms=['Model_T', 'S.U.V.', 'SUV', 'Stanley_Steamer', 'ambulance', 'beach_waggon', 'beach_wagon', 'bus', 'cab', 'compact', 'compact_car', 'convertible', 'coupe', 'cruiser', 'electric', 'electric_automobile', 'electric_car', 'estate_car', 'gas_guzzler', 'hack', 'hardtop', 'hatchback', 'heap', 'horseless_carriage', 'hot-rod', 'hot_rod', 'jalopy', 'jeep', 'landrover', 'limo', 'limousine', 'loaner', 'minicar', 'minivan', 'pace_car', 'patrol_car', 'phaeton', 'police_car', 'police_cruiser', 'prowl_car', 'race_car', 'racer', 'racing_car', 'roadster', 'runabout', 'saloon', 'secondhand_car', 'sedan', 'sport_car', 'sport_utility', 'sport_utility_vehicle', 'sports_car', 'squad_car', 'station_waggon', 'station_wagon', 'stock_car', 'subcompact', 'subcompact_car', 'taxi', 'taxicab', 'tourer', 'touring_car', 'two-seater', 'used-car', 'waggon', 'wagon']


In [10]:
def hyponyms_rec(synset) -> Set:
    hs = synset.hyponyms()
    res = set(hs)
    for h in hs:
        res |= hyponyms_rec(h)
    return res


ss = wn.synsets("car")
for s in ss:
    tmp = []
    for synset in hyponyms_rec(s):
        for lm in synset.lemma_names():
            tmp.append(lm)
    tmp.sort()
    print(f"{s.definition()}\n{len(tmp)} hyponyms={tmp}\n\n")

a motor vehicle with four wheels; usually propelled by an internal combustion engine
75 hyponyms=['Model_T', 'S.U.V.', 'SUV', 'Stanley_Steamer', 'ambulance', 'beach_waggon', 'beach_wagon', 'berlin', 'brougham', 'bus', 'cab', 'compact', 'compact_car', 'convertible', 'coupe', 'cruiser', 'electric', 'electric_automobile', 'electric_car', 'estate_car', 'finisher', 'funny_wagon', 'gas_guzzler', 'gypsy_cab', 'hack', 'hardtop', 'hatchback', 'heap', 'horseless_carriage', 'hot-rod', 'hot_rod', 'jalopy', 'jeep', 'landrover', 'limo', 'limousine', 'loaner', 'minicab', 'minicar', 'minivan', 'pace_car', 'panda_car', 'patrol_car', 'phaeton', 'police_car', 'police_cruiser', 'prowl_car', 'race_car', 'racer', 'racing_car', 'roadster', 'runabout', 'saloon', 'secondhand_car', 'sedan', 'shooting_brake', 'sport_car', 'sport_utility', 'sport_utility_vehicle', 'sports_car', 'squad_car', 'station_waggon', 'station_wagon', 'stock_car', 'stock_car', 'subcompact', 'subcompact_car', 'taxi', 'taxicab', 'tourer', 't

In [11]:
ss = wn.synsets("ape")
for s in ss:
    tmp = []
    for synset in hyponyms_rec(s):
        for lm in synset.lemma_names():
            tmp.append(lm)
    tmp.sort()
    print(f"{s.definition()}\n{len(tmp)} hyponyms={tmp}\n\n")

any of various primates with short tails or no tail at all
34 hyponyms=['Gorilla_gorilla', 'Gorilla_gorilla_beringei', 'Gorilla_gorilla_gorilla', 'Gorilla_gorilla_grauri', 'Hylobates_lar', 'Hylobates_syndactylus', 'Pan_paniscus', 'Pan_troglodytes', 'Pan_troglodytes_schweinfurthii', 'Pan_troglodytes_troglodytes', 'Pan_troglodytes_verus', 'Pongo_pygmaeus', 'Symphalangus_syndactylus', 'anthropoid_ape', 'bonobo', 'central_chimpanzee', 'chimp', 'chimpanzee', 'eastern_chimpanzee', 'eastern_lowland_gorilla', 'gibbon', 'gorilla', 'great_ape', 'lesser_ape', 'mountain_gorilla', 'orang', 'orangutan', 'orangutang', 'pongid', 'pygmy_chimpanzee', 'siamang', 'silverback', 'western_chimpanzee', 'western_lowland_gorilla']


someone who copies the words or behavior of another
3 hyponyms=['epigon', 'epigone', 'parrot']


person who resembles a nonhuman primate
0 hyponyms=[]


imitate uncritically and in every aspect
0 hyponyms=[]


represent in or produce a caricature of
0 hyponyms=[]




In [12]:
ss = wn.synsets("bank")
for s in ss:
    tmp = []
    for synset in hyponyms_rec(s):
        for lm in synset.lemma_names():
            tmp.append(lm)
    tmp.sort()
    print(f"{s.definition()}\n{len(tmp)} hyponyms={tmp}\n\n")

sloping land (especially the slope beside a body of water)
3 hyponyms=['riverbank', 'riverside', 'waterside']


a financial institution that accepts deposits and channels the money into lending activities
24 hyponyms=['FSB', 'Federal_Reserve_Bank', 'Home_Loan_Bank', 'MSB', 'acquirer', 'acquirer', 'agent_bank', 'agent_bank', 'building_society', 'commercial_bank', 'credit_union', 'federal_savings_bank', 'full_service_bank', 'lead_bank', 'member_bank', 'merchant_bank', 'mutual_savings_bank', 'national_bank', 'reserve_bank', 'savings_and_loan', 'savings_and_loan_association', 'savings_bank', 'state_bank', 'thrift_institution']


a long ridge or pile
3 hyponyms=['bluff', 'sandbank', 'shoal']


an arrangement of similar objects in a row or in tiers
0 hyponyms=[]


a supply or stock held in reserve for future use (especially in emergencies)
4 hyponyms=['blood_bank', 'eye_bank', 'food_bank', 'soil_bank']


the funds held by a gambling house or the dealer in some gambling games
0 hyponyms=[]




# Hypernyms

In [13]:
motorcar.hypernyms()

[Synset('motor_vehicle.n.01')]

In [14]:
class Word(NamedTuple):
    max_depth: int
    min_depth: int
    name: str
    lemma: str



# Get the path(s) from this synset to the root,
# where each path is a list of the synset nodes traversed on the way to the root.
for i, p in enumerate(motorcar.hypernym_paths()):
    words = []
    for synset in p:
        for lemma in synset.lemmas():
            words.append(
                Word(
                    name=synset.name(),
                    lemma=lemma.name(),
                    max_depth=synset.max_depth(),
                    min_depth=synset.min_depth()
                )
            )
    words.sort()
    print(f"i={i}\nwords={words}")

i=0
words=[Word(max_depth=0, min_depth=0, name='entity.n.01', lemma='entity'), Word(max_depth=1, min_depth=1, name='physical_entity.n.01', lemma='physical_entity'), Word(max_depth=2, min_depth=2, name='object.n.01', lemma='object'), Word(max_depth=2, min_depth=2, name='object.n.01', lemma='physical_object'), Word(max_depth=3, min_depth=3, name='whole.n.02', lemma='unit'), Word(max_depth=3, min_depth=3, name='whole.n.02', lemma='whole'), Word(max_depth=4, min_depth=4, name='artifact.n.01', lemma='artefact'), Word(max_depth=4, min_depth=4, name='artifact.n.01', lemma='artifact'), Word(max_depth=5, min_depth=5, name='instrumentality.n.03', lemma='instrumentality'), Word(max_depth=5, min_depth=5, name='instrumentality.n.03', lemma='instrumentation'), Word(max_depth=6, min_depth=6, name='container.n.01', lemma='container'), Word(max_depth=8, min_depth=7, name='wheeled_vehicle.n.01', lemma='wheeled_vehicle'), Word(max_depth=9, min_depth=8, name='self-propelled_vehicle.n.01', lemma='self-prop

In [15]:
print(dir(wn.synsets("motorcar")[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_all_hypernyms', '_definition', '_examples', '_frame_ids', '_hypernyms', '_instance_hypernyms', '_iter_hypernym_lists', '_lemma_names', '_lemma_pointers', '_lemmas', '_lexname', '_max_depth', '_min_depth', '_name', '_needs_root', '_offset', '_pointers', '_pos', '_related', '_shortest_hypernym_paths', '_wordnet_corpus_reader', 'acyclic_tree', 'also_sees', 'attributes', 'causes', 'closure', 'common_hypernyms', 'definition', 'entailments', 'examples', 'frame_ids', 'hypernym_distances', 'hypernym_paths', 'hypernyms', 'hyponyms', 'in_region_domains', 'in_topic_domains', 'in_usage_domains', 'instance_hypernyms', 'instance_hyponyms', 'jcn_s

In [16]:
synsets = wn.synset("right_whale.n.01").lowest_common_hypernyms(wn.synset("minke_whale.n.01"))
sim = wn.synset("right_whale.n.01").path_similarity(wn.synset("minke_whale.n.01"))
print(f"sim={sim}")
for s in synsets:
    print(f"{s}, min_depth={s.min_depth()}")

sim=0.25
Synset('baleen_whale.n.01'), min_depth=14


In [17]:
synsets = wn.synset("right_whale.n.01").lowest_common_hypernyms(wn.synset("orca.n.01"))
sim = wn.synset("right_whale.n.01").path_similarity(wn.synset("orca.n.01"))
print(f"sim={sim}")
for s in synsets:
    print(f"{s}, min_depth={s.min_depth()}")

sim=0.16666666666666666
Synset('whale.n.02'), min_depth=13


In [18]:
synsets = wn.synset("right_whale.n.01").lowest_common_hypernyms(wn.synset("tortoise.n.01"))
sim = wn.synset("right_whale.n.01").path_similarity(wn.synset("tortoise.n.01"))
print(f"sim={sim}")
for s in synsets:
    print(f"{s}, min_depth={s.min_depth()}")

sim=0.07692307692307693
Synset('vertebrate.n.01'), min_depth=8


In [19]:
synsets = wn.synset("right_whale.n.01").lowest_common_hypernyms(wn.synset("novel.n.01"))
sim = wn.synset("right_whale.n.01").path_similarity(wn.synset("novel.n.01"))
print(f"sim={sim}")
for s in synsets:
    print(f"{s}, min_depth={s.min_depth()}")

sim=0.043478260869565216
Synset('entity.n.01'), min_depth=0


In [20]:
def sim(left, right):
    res = 0
    for a in wn.synsets(left):
        for b in wn.synsets(right):
            res = max(res, a.path_similarity(b))
    return res


print(f"sim(car, automobile)={sim('car', 'automobile')}")
print(f"sim(cat, box)={sim('cat', 'box')}")
print(f"sim(apple, pear)={sim('apple', 'pear')}")

sim(car, automobile)=1.0
sim(cat, box)=0.16666666666666666
sim(apple, pear)=0.3333333333333333


# Polysemy
The polysemy of a word is the number of senses it has.

In [21]:
len(wn.synsets("dog", "n"))

7

In [22]:
len(wn.synsets("bank", "n"))

10

In [23]:
context = "I went to the bank to deposit money".split()
s = lesk(context, "bank", "n")
print(f"{s}, definition={s.definition()}")

Synset('savings_bank.n.02'), definition=a container (usually with a slot in the top) for keeping money at home


In [24]:
context = "The plane banked sharply to the left to avoid the coming storm".split()
s = lesk(context, "bank", "v")
print(f"{s}, definition={s.definition()}")

Synset('bank.v.07'), definition=cover with ashes so to control the rate of burning


In [25]:
context = "follow the river bank to the estuary".split()
s = lesk(context, "bank", "n")
print(f"{s}, definition={s.definition()}")

Synset('bank.n.07'), definition=a slope in the turn of a road or track; the outside is higher than the inside in order to reduce the effects of centrifugal force


In [26]:
context = "Investors were banking on the company's growth".split()
s = lesk(context, "bank", "v")
print(f"{s}, definition={s.definition()}")

Synset('bank.v.05'), definition=be in the banking business


In [27]:
nltk.word_tokenize("foo-bar (foo) bar!?")

['foo-bar', '(', 'foo', ')', 'bar', '!', '?']

In [28]:
tokens = nltk.word_tokenize("I went to the bank to deposit money")
nltk.pos_tag(tokens)

[('I', 'PRP'),
 ('went', 'VBD'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('bank', 'NN'),
 ('to', 'TO'),
 ('deposit', 'VB'),
 ('money', 'NN')]

In [29]:
synsets = list(wn.all_synsets())
print(f"len(synsets)={len(synsets)}")

len(synsets)=117659


In [30]:
n = 0
definitions = {}
for s in synsets:
    d = s.definition().strip()
    if len(d) == 0:
        n += 1
        continue
    definitions[s.name()] = d
print(f"{n} synsets have no definition")
print(f"len(definitions)={len(definitions)}")
assert len(synsets) == len(definitions)

0 synsets have no definition
len(definitions)=117659
